In [160]:
import json
import re
from pprint import pprint

import numpy as np
import pandas as pd
from scipy import stats
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
import sklearn.model_selection
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB

In [161]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/jupyter/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [162]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
from nltk.stem import SnowballStemmer
stemmer = SnowballStemmer(language = 'russian')

In [163]:
stop = stopwords.words('russian')
# удаляем из списка стоп слов  'не'
stop.remove('не')

# функция удаления стоп слов 
def replace_stop(string):
    mas = string.split()
    for i in stop:
        while i in mas :
            mas.remove(i)
    string  = ' '.join(mas)
    return string

# Функция приведения слова к нормальному виду(лемматизация  слова)
def norm(s):
    s = s.split()
    s = map(lambda x: morph.parse(x)[0].normal_form,s)
    return ' '.join(s)

# Удаление стоп слов и знаков препинаия,нижний регистр, нормальный вид
def clean(data, column):
    patt = re.compile("[^\w\s]")
    data.loc[:, column] = data[column].str.replace(
    patt, " ", regex=True
    ).map(str.lower).apply(replace_stop).apply(norm)
    
    return data
#  векторнизация предложений
def vector(data):
    vectorizer = CountVectorizer()
    data = vectorizer.fit_transform(data)
    data = data.toarray()
    
    return data
    
    

# Train

## Загрузка данных и анализ

In [164]:
train = pd.read_csv("/home/jupyter/mnt/s3/hse-data-vladimir/InternationaLiders/train.csv", index_col = 0)
print(f"Размер: {train.shape=}\n")
print("Типы столбцов: ")
display(train.dtypes)
print("Фрагмент данных: ")
display(train.head())


Размер: train.shape=(19361, 4)

Типы столбцов: 


sentence     object
1category    object
2category    object
sentiment    object
dtype: object

Фрагмент данных: 


,sentence,1category,2category,sentiment
4754,При этом всегда получал качественные услуги.,Communication,NaN,+
4417,"Не вижу, за что хотя бы 2 поставить, сервис на 1!",?,NaN,−
3629,"Вот так ""Мой любимый"" банк МКБ меня обманул.",?,NaN,−
11640,Отвратительное отношение к клиентам.,Communication,NaN,−
5571,"Всегда в любое время дня и ночи помогут, ответ...",Communication,NaN,+


## И тут хочется уточнить нашу задачу
Перед нами стоит задача показать хорошее предсказание как на трейне, так и на тесте. НО, по своей природе эти датасеты отличаются. В трейне первая категория может быть не определена (что нормально, ведь отзыв может быть "Круто, приятно узнать это сегодня."). А тест имеет только предложения, которые имеют ту или иную семинатику. Поэтому специльно для трейна мы будем обрабатывать первую категорию с вопросами, а длятеста мы обучим модель без категории с вопросами.

## Предобработка

In [165]:
# Удаление стоп слов и знаков препинаия,нижний регистр, нормальный вид
train  = clean(train, 'sentence')
train.head()

,sentence,1category,2category,sentiment
4754,получать качественный услуга,Communication,NaN,+
4417,не видеть хотя 2 поставить сервис 1,?,NaN,−
3629,любимый банк мкб обмануть,?,NaN,−
11640,отвратительный отношение клиент,Communication,NaN,−
5571,любой время день ночь помочь ответить решить,Communication,NaN,+


 ### Векторизация и разделение на выборки

In [63]:
# Векторизация 
X,y = train['sentence'], train['sentiment']
X = vector(X)

# Раздление на тетс и трейн
X_train, X_test, y_train, y_test = train_test_split(
        X, y,
        train_size=0.8,
        test_size=0.2,
        random_state=42
        )

## Модели 

## GNB

In [64]:
gnb = GaussianNB()
gnb.fit(X_train, y_train)
prediction_test_targets = gnb.predict(X_test)
print('accuracy ',accuracy_score(y_test, prediction_test_targets))
print('roc ', roc_auc_score(y_test, gnb.predict_proba(X_test), multi_class = 'ovr'))

accuracy  0.6421378776142526
roc  0.7657203869452193


## MLP

In [ ]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier()
mlp.fit(X_train, y_train)
prediction_test_targets = mlp.predict(X_test)
print('accuracy ',accuracy_score(y_test, prediction_test_targets))
print('roc ', roc_auc_score(y_test, mlp.predict_proba(X_test), multi_class = 'ovr'))

## ЛОГ

In [65]:
from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression()
classifier.fit(X_train, y_train)
prediction_test_targets = classifier.predict(X_test)
print('accuracy ',accuracy_score(y_test, prediction_test_targets))
print('roc ', roc_auc_score(y_test, classifier.predict_proba(X_test), multi_class = 'ovr'))

accuracy  0.8631551768654789
roc  0.9366415784663644


/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:938: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


## MNB

In [66]:
from sklearn.naive_bayes import MultinomialNB

clf = MultinomialNB(alpha=1)
clf.fit(X_train, y_train)
prediction_test_targets = clf.predict(X_test)
print('accuracy ',accuracy_score(y_test, prediction_test_targets))
print('roc ', roc_auc_score(y_test, clf.predict_proba(X_test), multi_class = 'ovr'))

accuracy  0.8288148721920992
roc  0.928355805644398


Логистическая регерссия дала лучшие результаты

# Output Train

###  Таблица веротятностей для test 

In [39]:
pd.DataFrame(data = classifier.predict_proba(X_test), index = y_test.index, columns = classifier.classes_ )

,+,?,−
17308,0.001732,0.012234,0.986034
17426,0.029709,0.068030,0.902261
7006,0.911916,0.086876,0.001208
18169,0.002589,0.009109,0.988302
21247,0.400486,0.009275,0.590239
...,...,...,...
11604,0.078505,0.027463,0.894032
4929,0.982485,0.017066,0.000449
9798,0.058597,0.034099,0.907304
21239,0.002646,0.010237,0.987116


In [45]:
print('accuracy ',accuracy_score(y_test, prediction_test_targets))
print('roc ', roc_auc_score(y_test, classifier.predict_proba(X_test), multi_class = 'ovr'))

accuracy  0.8424993545055512
roc  0.9365729609640843


### Таблица веротяностей для всего файла Train

In [41]:
input_train = pd.DataFrame(data = classifier.predict_proba(X), index = y, columns = classifier.classes_ )
input_train 

,+,?,−
sentiment,,,
+,0.483110,0.317137,0.199753
−,0.008340,0.018662,0.972998
−,0.074755,0.060398,0.864847
−,0.010254,0.007641,0.982105
+,0.566099,0.356588,0.077313
...,...,...,...
−,0.004878,0.055871,0.939251
−,0.372061,0.040305,0.587635
+,0.619880,0.328592,0.051528


In [44]:
print('accuracy ',accuracy_score(y, classifier.predict(X)))
print('roc ', roc_auc_score(y, classifier.predict_proba(X), multi_class = 'ovr'))

accuracy  0.9188058468054336
roc  0.9779945597767258


# Output Test

### Загрузка

In [171]:
test = pd.read_csv("/home/jupyter/mnt/s3/hse-data-vladimir/InternationaLiders/1sentencenewtest.csv", index_col = 0)
print(f"Размер: {test.shape=}\n")
print("Типы столбцов: ")
display(test.dtypes)
print("Фрагмент данных: ")
display(test.head())

Размер: test.shape=(949, 1)

Типы столбцов: 


0    object
dtype: object

Фрагмент данных: 


,0
0,15.03.2022 обратился на горячую линию для закр...
1,"Уже который год в ТКБ не решается ""глобальная ..."
2,Добрый день
3,"Добрый день Сегодня, зайдя в свой личный кабин..."
4,"Обслуживаюсь в Тинькофф пару лет, возникла жес..."


Обработка

In [172]:
test = clean(test, '0')
test.head()

,0
0,15 03 2022 обратиться горячий линия закрытие с...
1,который год ткб не решаться глобальный проблем...
2,добрый день
3,добрый день сегодня зайти свой личный кабинет ...
4,обслуживаться тинькофф пара год возникнуть жес...


Векторизация

In [173]:
X_train, y_train = train['sentence'], train["sentiment"]
X_test = test['0']

vectorizer = CountVectorizer()
vec = vectorizer.fit(pd.concat([X_train,X_test]))
X_train = vectorizer.transform(X_train).toarray()
X_test = vectorizer.transform(X_test).toarray()

Модель

In [174]:
classifier = LogisticRegression()
classifier.fit(X_train, y_train)
proba = classifier.predict_proba(X_test)

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:938: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [175]:
classifier.classes_

array(['+', '?', '−'], dtype=object)

In [178]:
input_test = pd.DataFrame(data = proba, index = test.index, columns = classifier.classes_ )
input_test = pd.merge(input_test, test, left_index = True, right_index = True, how = 'inner')
input_test = input_test[['0', '+',  '−', '?' ]]
input_test

,0,+,−,?
0,15 03 2022 обратиться горячий линия закрытие с...,0.103934,0.711518,0.184548
1,который год ткб не решаться глобальный проблем...,0.847510,0.082847,0.069643
2,добрый день,0.409348,0.153404,0.437249
3,добрый день сегодня зайти свой личный кабинет ...,0.047824,0.415146,0.537030
4,обслуживаться тинькофф пара год возникнуть жес...,0.956751,0.000497,0.042751
...,...,...,...,...
944,отвратительный сервис отношение клиент являтьс...,0.041220,0.928667,0.030113
945,28 04 2022 обратиться банк возможность перевод...,0.475327,0.364548,0.160126
946,начало 2021 год акция выплата 8 кешбек оплата ...,0.179557,0.608863,0.211580
947,бездействие банк некомпетентность сотрудник,0.006099,0.985956,0.007945


In [ ]:
input_test.to_csv("demo.csv", index=True)